In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import pickle
import os

In [2]:
# Function to make 4 classifications based on balance due and the portion of the penality paid
def make_classification(penality, paid, due):
    if due == 0:
        if penality > paid:
            classification = 0
            desc = 'NO BALANCE - PARITALLY PAID'
        else:
            classification = 1
            desc = 'NO BALANCE - PAID IN FULL'
    else:
        if paid == 0:
            classification = 2
            desc = 'BALANCE DUE - UNPAID'
        else:
            classification = 3
            desc = 'BALANCE DUE - PARITALLY PAID'
    return classification, desc

# convert values to float
def make_float(expected_float):
    try:
        if type(expected_float) == str:
            expected_float = expected_float.replace('+', '').replace(',', '')
        return float(expected_float)
    except:
        # print expected_float
        return np.nan
    


In [3]:
# Trump owned buildings identified from:
# https://www.bisnow.com/new-york/news/economy/how-big-is-trumps-nyc-empire-63995?single-page

# BIN NUMBER
_40_WALL_ST = '1001018' # TRUMP_BUILDING
_1290_AVENUE_OF_THE_AMERICAS = '1034510' # _1290_AVENUE_OF_THE_AMERICAS 30% Stake in Vornado owned building
_1_CENTRAL_PARK_WEST = '1027191' # TRUMP_INTERNATIONAL_HOTEL_AND_TOWER
_725_5TH_AVE = '1035794' # TRUMP_TOWER
_246_SPRING_ST = '1088431' # TRUMP_SOHO
_502_PARK_AVE = '1040756' # TRUMP_PARK_AVE
_327_E_47TH_ST = '1038908' # TRUMP_WORLD_TOWER
_200_E_69TH_ST = '1043902' # TRUMP_PALACE
_106_CENTRAL_PARK_S = '1069595' # TRUMP_PARC
_610_PARK_AVE = '1041086' # _610_PARK_AVE

TRUMP_BUILDINGS = [_106_CENTRAL_PARK_S, _1290_AVENUE_OF_THE_AMERICAS, 
                   _1_CENTRAL_PARK_WEST, _200_E_69TH_ST, _246_SPRING_ST, 
                   _327_E_47TH_ST, _40_WALL_ST, _502_PARK_AVE,
                   _610_PARK_AVE, _725_5TH_AVE]
# Wollman Rink - Central Park (Not a building)

In [4]:
# DOB ECB BUILDING VIOLATION DATA
DOB_ECB = 'data/20170322_DOB_ECB_Violations.csv'
ACS_5YR_RACE = 'data/CENSUS_TRACT_RACE_INCOME/ACS_15_5YR_DP05_with_ann.csv'
ACS_5YR_INCOME = 'data/CENSUS_TRACT_RACE_INCOME/ACS_15_5YR_S1901_with_ann.csv'
PLUTO_BX = 'data/PLUTO/Bronx/BXMapPLUTO.shp'
PLUTO_BK = 'data/PLUTO/Brooklyn/BKMapPLUTO.shp'
PLUTO_QN = 'data/PLUTO/Queens/QNMapPLUTO.shp'
PLUTO_MN = 'data/PLUTO/Manhattan/MNMapPLUTO.shp'
PLUTO_SI = 'data/PLUTO/Staten_Island/SIMapPLUTO.shp'
MASTER_PLUTO_PICKLE = 'processed_data/master_pluto.pickle'

In [5]:
DF_DOB_ECB = pd.read_csv(DOB_ECB, usecols=['BIN', 'ISSUE_DATE', 'SEVERITY', 
                                           'PENALITY_IMPOSED', 'AMOUNT_PAID', 
                                           'BALANCE_DUE', 'ECB_VIOLATION_STATUS', 
                                           'BORO', 'BLOCK', 'LOT', 'VIOLATION_DESCRIPTION',
                                           'INFRACTION_CODE1'],
                         dtype={'BIN': str, 'PENALITY_IMPOSED': float,
                                'AMOUNT_PAID': float, 'BALANCE_DUE': float}) 

/Users/Nate/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Only 181 FINES associated with trump buildings

print '"TRUMP PROPERTIES" WITH FINES', len(DF_DOB_ECB[(DF_DOB_ECB['PENALITY_IMPOSED'] > 0) & (DF_DOB_ECB['BIN'].isin(TRUMP_BUILDINGS))])
print 'PENALITY IMPOSED', DF_DOB_ECB[(DF_DOB_ECB['PENALITY_IMPOSED'] > 0) & (DF_DOB_ECB['BIN'].isin(TRUMP_BUILDINGS))]['PENALITY_IMPOSED'].sum()
print 'AMOUNT_PAID', DF_DOB_ECB[(DF_DOB_ECB['PENALITY_IMPOSED'] > 0) & (DF_DOB_ECB['BIN'].isin(TRUMP_BUILDINGS))]['AMOUNT_PAID'].sum()
print 'OUTSTANDING BALANCE', DF_DOB_ECB[(DF_DOB_ECB['PENALITY_IMPOSED'] > 0) & (DF_DOB_ECB['BIN'].isin(TRUMP_BUILDINGS))]['BALANCE_DUE'].sum()

"TRUMP PROPERTIES" WITH FINES 181
PENALITY IMPOSED 183425.0
AMOUNT_PAID 177357.27
OUTSTANDING BALANCE 765.34


## Perhaps fines associated with trump owned buildings may not hold enough data

In [7]:
# VIOLATIONS WITH A BALANCE DUE
DF_DOB_ECB[DF_DOB_ECB['BALANCE_DUE'] > 0]['BALANCE_DUE'].sum()

748325710.25

In [8]:
# TOTAL PENALITIES VALUE
DF_DOB_ECB['PENALITY_IMPOSED'].sum()

1669745657.52

In [9]:
# FINES PAID
DF_DOB_ECB['AMOUNT_PAID'].sum()

616422508.6099999

In [10]:
DF_DOB_ECB_FINES = DF_DOB_ECB[DF_DOB_ECB['PENALITY_IMPOSED'] > 0].copy()

In [11]:
DF_DOB_ECB_FINES['CLASSIFICATION'], DF_DOB_ECB_FINES['CLASS_DESC'] = np.vectorize(make_classification)(DF_DOB_ECB_FINES['PENALITY_IMPOSED'],
                                                                                                       DF_DOB_ECB_FINES['AMOUNT_PAID'],
                                                                                                       DF_DOB_ECB_FINES['BALANCE_DUE'])

In [12]:
# Breakdown of different classes
DF_DOB_ECB_FINES[['CLASS_DESC', 'ECB_VIOLATION_STATUS']].groupby('CLASS_DESC').count()

,ECB_VIOLATION_STATUS
CLASS_DESC,
BALANCE DUE - PARITALLY PAID,31699
BALANCE DUE - UNPAID,104709
NO BALANCE - PAID IN FULL,479941
NO BALANCE - PARITALLY PAID,136943


In [35]:
# race data
ACS_5YR_RACE_DF = pd.read_csv(ACS_5YR_RACE,skiprows=[1], usecols=['GEO.id2', 'HC01_VC03', 'HC01_VC49', 'HC01_VC50', 'HC01_VC51','HC01_VC56', 'HC01_VC64', 'HC01_VC69', 'HC01_VC23'])

# rename columns
ACS_5YR_RACE_DF.rename(columns={'HC01_VC03': 'TOTAL_POPULATION', 'HC01_VC49': 'WHITE',
                        'HC01_VC50': 'BLACK_AFRICAN_AMERICAN', 'HC01_VC51': 'AMERICAN_INDIAN_AND_ALASKA_NATIVE',
                        'HC01_VC56': 'ASIAN', 'HC01_VC64': 'NATIVE_HAWAIIAN_AND_OTHER_PACIFIC_ISLANDER',
                        'HC01_VC69': 'SOME_OTHER_RACE', 'HC01_VC23': 'MEDIAN_AGE', 'GEO.id2': 'GEOID'}, inplace=True)

for i in ACS_5YR_RACE_DF.columns[ACS_5YR_RACE_DF.columns!='GEOID']:
    ACS_5YR_RACE_DF[i] = ACS_5YR_RACE_DF[i].apply(lambda x: make_float(x))

ACS_5YR_RACE_DF['GEOID'] = ACS_5YR_RACE_DF['GEOID'].astype(str)

# income data
ACS_5YR_INCOME_DF = pd.read_csv(ACS_5YR_INCOME,skiprows=[1],usecols=['GEO.id2', 'HC01_EST_VC01', 'HC01_EST_VC15'])

# rename columns
ACS_5YR_INCOME_DF.rename(columns={'HC01_EST_VC01': 'TOTAL_HOUSEHOLDS', 
                       'HC01_EST_VC15': 'MEAN_INCOME', 'GEO.id2': 'GEOID'}, inplace=True)

#convert values to float
for i in ACS_5YR_INCOME_DF.columns[ACS_5YR_INCOME_DF.columns!='GEOID']:
    ACS_5YR_INCOME_DF[i] = ACS_5YR_INCOME_DF[i].apply(lambda x: make_float(x))

ACS_5YR_INCOME_DF['GEOID'] = ACS_5YR_RACE_DF['GEOID'].astype(str)


In [46]:
def import_filter_pluto():
    
    # import PLUTO for 5 boros
    BK = gp.read_file(PLUTO_BK)
    BX = gp.read_file(PLUTO_BX)
    MN = gp.read_file(PLUTO_MN)
    QN = gp.read_file(PLUTO_QN)
    SI = gp.read_file(PLUTO_SI)
    
    # merge 5 boro PLUTO datasets 
    pluto_agg = BK.append(BX)
    pluto_agg = pluto_agg.append(MN)
    pluto_agg = pluto_agg.append(QN)
    pluto_agg = pluto_agg.append(SI)
    
    pluto_select = pluto_agg[['BBL','YearBuilt','Tract2010','UnitsRes',
    'BldgClass','LandUse','BldgArea',
    'ComArea',
    'ResArea',
    'UnitsTotal',
    'AssessTot',
    'BuiltFAR','LotArea','OwnerType']]
    with open(MASTER_PLUTO_PICKLE, 'wb') as handle:
        pickle.dump(pluto_select, handle, protocol=pickle.HIGHEST_PROTOCOL)

### CLEANING PLUTO

In [47]:
if os.path.exists(MASTER_PLUTO_PICKLE):
    print "File exists. Loading pickle..."
    # load pickle of PLUTO data
    with open(MASTER_PLUTO_PICKLE, 'rb') as handle:
        master_pluto = pickle.load(handle)
    
else:
    print "File does not yet exist. Importing and filtering PLUTO. This could take several minutes..."
    # first time only, import, filter, and save processed PLUTO as a pickle for future use
    import_filter_pluto()
    
    # load pickle of PLUTO data
    with open(MASTER_PLUTO_PICKLE, 'rb') as handle:
        master_pluto = pickle.load(handle)

File exists. Loading pickle...


In [48]:
boro_to_ct = {'1':'36061','2':'36005','3':'36047','4':'36081','5':'36085'}

master_pluto['ST_CT_FIPS'] = master_pluto['BBL'].apply(lambda x: boro_to_ct[str(x)[0]])
master_pluto['Tract2010'] = master_pluto['Tract2010'].apply(lambda x: x + '00' if len(x) == 4 else x)

In [51]:
master_pluto['BBL'] = master_pluto['BBL'].astype(int)
master_pluto['BBL'] = master_pluto['BBL'].astype(str)

In [55]:
master_pluto.reset_index()['BBL'][0]

'3000060010'

In [56]:
DF_DOB_ECB_FINES[]

,ECB_VIOLATION_STATUS,BIN,BORO,BLOCK,LOT,ISSUE_DATE,SEVERITY,VIOLATION_DESCRIPTION,PENALITY_IMPOSED,AMOUNT_PAID,BALANCE_DUE,INFRACTION_CODE1,CLASSIFICATION,CLASS_DESC
0,RESOLVE,4080791,4,3388,22.0,20100129,Unknown,FAILURE TO COMPLY EITH COMM ORDER TO FILE A CE...,4000.0,937.89,0.00,263,0,NO BALANCE - PARITALLY PAID
2,RESOLVE,2003818,2,2568,57.0,20090526,Unknown,97X10.97X10 IRCNY 11-02 ONLY ELEVATOR IN BLDG ...,5000.0,1000.00,0.00,151,0,NO BALANCE - PARITALLY PAID
3,RESOLVE,1004967,1,398,18.0,20080527,Non-Hazardous,WORK DOES NOT CONFORM TO APPROVED PLANS LAYOUT...,2500.0,250.00,0.00,B25,0,NO BALANCE - PARITALLY PAID
4,RESOLVE,1073089,1,1517,7501.0,19910625,Non-Hazardous,"75B5,15M7. -75 PERFORM REQ. 5YR TESTS, -15 REP...",350.0,350.00,0.00,BP7,1,NO BALANCE - PAID IN FULL
5,RESOLVE,4268676,4,12398,227.0,19921022,Non-Hazardous,OCCUPANCY OF GARAGE CONTRARY TO THAT ALLOWED B...,175.0,175.00,0.00,B03,1,NO BALANCE - PAID IN FULL
6,RESOLVE,4180026,4,8798,36.0,20090521,Unknown,FAILURE TO SAFEGUARD ALL PERSONS AND PROPERTY ...,2400.0,2720.75,0.00,109,1,NO BALANCE - PAID IN FULL
7,RESOLVE,4180719,4,8830,52.0,19890323,Non-Hazardous,WORK WITHOUT A PERMIT WORK NOTED CEMENT BLOCK ...,625.0,625.00,0.00,B04,1,NO BALANCE - PAID IN FULL
8,RESOLVE,4206710,4,9675,4.0,20010610,Hazardous,WORK W/O PERMIT.WRK.NOTED:CELLAR WALL ERECTED ...,800.0,800.00,0.00,BQ2,1,NO BALANCE - PAID IN FULL
12,RESOLVE,1014350,1,780,76.0,19940517,Non-Hazardous,FAILURE TO MAINTAIN EXTERIOR BUILDING WALL. DE...,100.0,100.00,0.00,B06,1,NO BALANCE - PAID IN FULL
13,ACTIVE,3205069,3,7465,52.0,20081111,Unknown,"B254,18A,18B,50A,50B,32A,50C,50D,50E. 50A REMO...",500.0,500.00,0.00,254,1,NO BALANCE - PAID IN FULL
